## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Albany,US,42.6001,-73.9662,51.57,37,100,4.00,overcast clouds
1,1,Maceio,BR,-9.6658,-35.7353,78.24,78,0,4.61,clear sky
2,2,Mehamn,NO,71.0357,27.8492,39.20,93,100,23.22,overcast clouds
3,3,Dikson,RU,73.5069,80.5464,2.44,95,100,14.45,overcast clouds
4,4,Hithadhoo,MV,-0.6000,73.0833,82.89,73,99,4.50,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Maceio,BR,-9.6658,-35.7353,78.24,78,0,4.61,clear sky
4,4,Hithadhoo,MV,-0.6000,73.0833,82.89,73,99,4.50,overcast clouds
6,6,Puerto Ayora,EC,-0.7393,-90.3518,76.95,83,100,6.58,overcast clouds
9,9,Vaini,TO,-21.2000,-175.2000,80.76,89,75,10.36,broken clouds
10,10,Selma,US,36.5708,-119.6121,80.10,33,99,6.60,overcast clouds
12,12,Atuona,PF,-9.8000,-139.0333,79.88,71,42,17.36,scattered clouds
19,19,Guadalupe Y Calvo,MX,26.1000,-106.9667,77.95,11,57,11.97,broken clouds
24,24,Barra Patuca,HN,15.8000,-84.2833,81.23,71,0,10.31,clear sky
27,27,Kavieng,PG,-2.5744,150.7967,85.24,67,94,15.10,overcast clouds
29,29,Cururupu,BR,-1.8283,-44.8683,75.25,93,66,2.68,broken clouds


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
666,False,False,False,False,False,False,False,False,False,False
671,False,False,False,False,False,False,False,False,False,False
676,False,False,False,False,False,False,False,False,False,False
677,False,False,False,False,False,False,False,False,False,False


In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
chosen_cities_df = preferred_cities_df.dropna()
chosen_cities_df.count()

City_ID                217
City                   217
Country                217
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Weather Description    217
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = chosen_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Maceio,BR,78.24,clear sky,-9.6658,-35.7353,
4,Hithadhoo,MV,82.89,overcast clouds,-0.6000,73.0833,
6,Puerto Ayora,EC,76.95,overcast clouds,-0.7393,-90.3518,
9,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,
10,Selma,US,80.10,overcast clouds,36.5708,-119.6121,
12,Atuona,PF,79.88,scattered clouds,-9.8000,-139.0333,
19,Guadalupe Y Calvo,MX,77.95,broken clouds,26.1000,-106.9667,
24,Barra Patuca,HN,81.23,clear sky,15.8000,-84.2833,
27,Kavieng,PG,85.24,overcast clouds,-2.5744,150.7967,
29,Cururupu,BR,75.25,broken clouds,-1.8283,-44.8683,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace=True)
hotel_df.count()

City                   217
Country                217
Max Temp               217
Weather Description    217
Lat                    217
Lng                    217
Hotel Name             217
dtype: int64

In [32]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))